# Анализ данных в финам

In [96]:
import pandas as pd
import pandas_datareader as web
from datetime import datetime
from finam.export import Exporter, Market, Timeframe
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
init_notebook_mode(connected=True)
import numpy as np
import matplotlib as plt
import datetime

In [21]:
rosn=web.DataReader('ROSN', 'moex', start='2017-12-01', end='2018-12-31')
rosn.head(5)

,BOARDID,SHORTNAME,SECID,NUMTRADES,VALUE,OPEN,LOW,HIGH,CLOSE,WAPRICE
TRADEDATE,,,,,,,,,,
2017-12-04,RPMA,Роснефть,ROSN,98,3.503776e+09,292.0,241.0320,292.70,246.8392,NaN
2017-12-05,RPMA,Роснефть,ROSN,102,3.585532e+09,292.7,14.5075,293.45,290.1500,NaN
2017-12-06,RPMA,Роснефть,ROSN,91,3.594788e+09,293.4,241.4470,293.40,290.9000,NaN
2017-12-07,RPMA,Роснефть,ROSN,97,3.211593e+09,292.4,241.6545,293.60,291.1500,NaN
2017-12-08,RPMA,Роснефть,ROSN,100,3.576409e+09,292.5,242.2770,293.50,291.9000,NaN


In [23]:
exporter = Exporter()
exporter.lookup(code='ROSN')

,name,code,market
id,,,
17273,Роснефть,ROSN,1
17265,ROSN,ROSN,3
17290,Роснефть,ROSN,8
19894,ROSN,ROSN,14
22428,ROSN(RTS-Standard),ROSN,38
17273,Роснефть,ROSN,200
419953,Rosneft,ROSN,620


In [43]:
start_date = datetime(2017, 12, 31)
end_date = datetime(2018, 12, 31)
rosn = exporter.lookup(code='ROSN',market=Market.SHARES)

display(sber)

data_daily = exporter.download(
    rosn.index[0], 
    market=Market.SHARES,
    start_date=start_date,
    end_date=end_date
)
data_daily.head()

,name,code,market
id,,,
17273,Роснефть,ROSN,1


,<OPEN>,<HIGH>,<LOW>,<CLOSE>,<VOL>
index,,,,,
2018-01-03,292.15,297.95,292.15,297.95,2231440
2018-01-04,299.00,308.25,297.60,307.00,5754500
2018-01-05,307.00,313.65,304.65,311.95,4236460
2018-01-09,312.55,316.25,311.15,315.10,4572510
2018-01-10,316.10,319.85,312.00,317.75,5560000


In [44]:
data_daily.columns = ['Open', 'High','Low','Close','Volume']
data_daily.head(5)

,Open,High,Low,Close,Volume
index,,,,,
2018-01-03,292.15,297.95,292.15,297.95,2231440
2018-01-04,299.00,308.25,297.60,307.00,5754500
2018-01-05,307.00,313.65,304.65,311.95,4236460
2018-01-09,312.55,316.25,311.15,315.10,4572510
2018-01-10,316.10,319.85,312.00,317.75,5560000


In [45]:
data_volume_month=data_daily.Volume.resample('m').sum()
data_volume_month

index
2018-01-31    120369350
2018-02-28     84799770
2018-03-31     93115390
2018-04-30    195115790
2018-05-31     95871390
2018-06-30     82423870
2018-07-31     58669390
2018-08-31     72208990
2018-09-30     63172060
2018-10-31     83610160
2018-11-30    107027800
2018-12-31     75386260
Freq: M, Name: Volume, dtype: int64

In [46]:
open_m=data_daily.Open.resample('M').first().values
open_m

array([292.15, 345.  , 326.95, 314.5 , 390.05, 384.2 , 394.5 , 418.5 ,
       435.7 , 495.95, 460.  , 431.55])

In [47]:
close_m=data_daily.Close.resample('M').last().values
low_m=data_daily.Low.resample('M').min().values
high_m=data_daily.High.resample('M').max().values

In [48]:
month=data_daily.index.strftime('%b').unique()
month

Index(['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct',
       'Nov', 'Dec'],
      dtype='object')

In [49]:
data_daily.to_period('M').index.unique()

PeriodIndex(['2018-01', '2018-02', '2018-03', '2018-04', '2018-05', '2018-06',
             '2018-07', '2018-08', '2018-09', '2018-10', '2018-11', '2018-12'],
            dtype='period[M]', name='index', freq='M')

In [50]:
moth_a = pd.DataFrame(
    {'month':month,'open':open_m,'close':close_m,'low':low_m,'high':high_m},
    index=data_daily.to_period('M').index.unique())
moth_a.head(3)

,month,open,close,low,high
index,,,,,
2018-01,Jan,292.15,343.2,292.15,359.30
2018-02,Feb,345.00,330.5,319.50,353.00
2018-03,Mar,326.95,313.9,305.00,327.85


In [51]:
rosn_m = data_daily.resample('m').agg({'Open':'first','Low':'min','High':'max','Close':'last'})
rosn_m

,Open,Low,High,Close
index,,,,
2018-01-31,292.15,292.15,359.30,343.20
2018-02-28,345.00,319.50,353.00,330.50
2018-03-31,326.95,305.00,327.85,313.90
2018-04-30,314.50,281.65,388.25,383.00
2018-05-31,390.05,374.25,408.05,385.30
2018-06-30,384.20,371.40,399.85,396.15
2018-07-31,394.50,388.35,419.00,419.00
2018-08-31,418.50,407.20,444.95,435.70
2018-09-30,435.70,432.40,495.00,493.00


In [52]:
trace = go.Ohlc(x=data_daily.index,
                open=data_daily['Open'],
                high=data_daily['High'],
                low=data_daily['Low'],
                close=data_daily['Close'])

data = [trace]
iplot(data)

In [72]:
month=rosn_m.index.strftime('%b-2017')

trace2 = go.Candlestick(
                x=month, 
                open=rosn_m['Open'],
                high=rosn_m['High'],
                low=rosn_m['Low'],
                close=rosn_m['Close'])

data2 = [trace2]
fig = go.Figure(data=data2)
iplot(fig)

In [54]:
#Пересобираем данные

rosn_week = exporter.download(
    id_ = rosn.index[0],
    market = Market.SHARES,
    start_date = datetime(2017, 12, 31),
    end_date = datetime(2018, 12, 31),
    timeframe = Timeframe.WEEKLY
)
rosn_week.head(10)

,<OPEN>,<HIGH>,<LOW>,<CLOSE>,<VOL>
index,,,,,
2018-01-01,292.15,313.65,292.15,311.95,12222400
2018-01-08,312.55,326.95,311.15,324.35,20182890
2018-01-15,325.00,333.35,312.95,328.00,29394910
2018-01-22,328.00,359.30,324.40,351.80,41393810
2018-01-29,353.00,356.80,337.25,339.90,27506010
2018-02-05,336.00,340.45,320.35,324.55,28248480
2018-02-12,328.20,333.80,322.75,324.20,23422490
2018-02-19,325.40,338.00,319.50,337.30,12858890
2018-02-26,341.35,345.85,314.60,319.50,20156080


In [55]:
rosn_week.columns = ['Open', 'High','Low','Close','Volume']
rosn_week.head(5)

,Open,High,Low,Close,Volume
index,,,,,
2018-01-01,292.15,313.65,292.15,311.95,12222400
2018-01-08,312.55,326.95,311.15,324.35,20182890
2018-01-15,325.00,333.35,312.95,328.00,29394910
2018-01-22,328.00,359.30,324.40,351.80,41393810
2018-01-29,353.00,356.80,337.25,339.90,27506010


In [58]:
data_volume_week=rosn_week.Volume.resample('w').sum()
data_volume_week.head(5)

index
2018-01-07    12222400
2018-01-14    20182890
2018-01-21    29394910
2018-01-28    41393810
2018-02-04    27506010
Freq: W-SUN, Name: Volume, dtype: int64

In [59]:
open_w=rosn_week.Open.resample('W').first().values
open_w

array([292.15, 312.55, 325.  , 328.  , 353.  , 336.  , 328.2 , 325.4 ,
       341.35, 321.65, 314.65, 321.8 , 313.8 , 314.5 , 318.4 , 317.9 ,
       339.  , 381.05, 389.5 , 402.05, 387.85, 376.55, 383.5 , 382.8 ,
       381.85, 390.  , 394.5 , 402.4 , 406.3 , 394.05, 400.4 , 412.55,
       430.1 , 426.5 , 429.25, 435.7 , 438.95, 442.65, 442.15, 495.95,
       490.55, 468.25, 469.  , 454.45, 462.2 , 472.2 , 422.35, 405.8 ,
       431.55, 436.75, 416.7 , 414.7 , 429.  ])

In [60]:
close_w=rosn_week.Close.resample('W').last().values
low_w=rosn_week.Low.resample('W').min().values
high_w=rosn_week.High.resample('W').max().values

In [67]:
week = rosn_week.index.strftime('%W').unique()
week

Index(['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12',
       '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24',
       '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36',
       '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48',
       '49', '50', '51', '52', '53'],
      dtype='object')

In [68]:
rosn_week.to_period('W').index.unique()

PeriodIndex(['2018-01-01/2018-01-07', '2018-01-08/2018-01-14',
             '2018-01-15/2018-01-21', '2018-01-22/2018-01-28',
             '2018-01-29/2018-02-04', '2018-02-05/2018-02-11',
             '2018-02-12/2018-02-18', '2018-02-19/2018-02-25',
             '2018-02-26/2018-03-04', '2018-03-05/2018-03-11',
             '2018-03-12/2018-03-18', '2018-03-19/2018-03-25',
             '2018-03-26/2018-04-01', '2018-04-02/2018-04-08',
             '2018-04-09/2018-04-15', '2018-04-16/2018-04-22',
             '2018-04-23/2018-04-29', '2018-04-30/2018-05-06',
             '2018-05-07/2018-05-13', '2018-05-14/2018-05-20',
             '2018-05-21/2018-05-27', '2018-05-28/2018-06-03',
             '2018-06-04/2018-06-10', '2018-06-11/2018-06-17',
             '2018-06-18/2018-06-24', '2018-06-25/2018-07-01',
             '2018-07-02/2018-07-08', '2018-07-09/2018-07-15',
             '2018-07-16/2018-07-22', '2018-07-23/2018-07-29',
             '2018-07-30/2018-08-05', '2018-08-06/2018-

In [69]:
week_df = pd.DataFrame(
    {'week':week,'open':open_w,'close':close_w,'low':low_w,'high':high_w},
    index=rosn_week.to_period('W').index.unique())
week_df.head(3)

,week,open,close,low,high
index,,,,,
2018-01-01/2018-01-07,01,292.15,311.95,292.15,313.65
2018-01-08/2018-01-14,02,312.55,324.35,311.15,326.95
2018-01-15/2018-01-21,03,325.00,328.00,312.95,333.35


In [71]:
rosn_w = data_daily.resample('w').agg({'Open':'first','Low':'min','High':'max','Close':'last'})
rosn_w.head(5)

,Open,Low,High,Close
index,,,,
2018-01-07,292.15,292.15,313.65,311.95
2018-01-14,312.55,311.15,326.95,324.35
2018-01-21,325.00,312.95,333.35,328.00
2018-01-28,328.00,324.40,359.30,351.80
2018-02-04,353.00,337.25,356.80,339.90


In [115]:
# Я долго думал,как объединить два графика,и в итоге путём перебора дошёл до этого. Я не знаю,почему он берёт отсчёт с 1950 по 
# 2049,но с 2017 по 2018 там есть скользящая средняя и ohlc
month=rosn_w.index.strftime('%b-2017')

trace = go.Ohlc(x=rosn_week.index,
                open=rosn_week['Open'],
                high=rosn_week['High'],
                low=rosn_week['Low'],
                close=rosn_week['Close'])


trace2 = go.Candlestick(
                x=week, 
                open=rosn_w['Open'],
                high=rosn_w['High'],
                low=rosn_w['Low'],
                close=rosn_w['Close'])

layout = go.Layout(
    xaxis = dict(
        rangeslider = dict(
            visible = False
        )
    )
)


data3 = [trace,trace2]
fig = go.Figure(data=data3)
iplot(fig)